# Wrangle

In [1]:
#imports
import pandas as pd
import numpy as np

# system
import sys
# sys.path.append("../data")
sys.path.append("util_")
import wrangle_

In [2]:
# lacd data
df = pd.read_csv("data/known_exploited_vulnerabilities.csv")
df.head()

,cveID,vendorProject,product,vulnerabilityName,dateAdded,shortDescription,requiredAction,dueDate,knownRansomwareCampaignUse,notes,cwes
0,CVE-2021-27104,Accellion,FTA,Accellion FTA OS Command Injection Vulnerability,2021-11-03,Accellion FTA contains an OS command injection...,Apply updates per vendor instructions.,2021-11-17,Known,NaN,NaN
1,CVE-2021-27102,Accellion,FTA,Accellion FTA OS Command Injection Vulnerability,2021-11-03,Accellion FTA contains an OS command injection...,Apply updates per vendor instructions.,2021-11-17,Known,NaN,NaN
2,CVE-2021-27101,Accellion,FTA,Accellion FTA SQL Injection Vulnerability,2021-11-03,Accellion FTA contains a SQL injection vulnera...,Apply updates per vendor instructions.,2021-11-17,Known,NaN,NaN
3,CVE-2021-27103,Accellion,FTA,Accellion FTA Server-Side Request Forgery (SSR...,2021-11-03,Accellion FTA contains a server-side request f...,Apply updates per vendor instructions.,2021-11-17,Known,NaN,NaN
4,CVE-2021-21017,Adobe,Acrobat and Reader,Adobe Acrobat and Reader Heap-based Buffer Ove...,2021-11-03,Acrobat Acrobat and Reader contain a heap-base...,Apply updates per vendor instructions.,2021-11-17,Unknown,NaN,NaN


In [3]:
# look at the shape
df.shape

(1140, 11)

In [4]:
#look at the columns
df.columns

Index(['cveID', 'vendorProject', 'product', 'vulnerabilityName', 'dateAdded',
       'shortDescription', 'requiredAction', 'dueDate',
       'knownRansomwareCampaignUse', 'notes', 'cwes'],
      dtype='object')

In [5]:
# check the full info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1140 entries, 0 to 1139
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   cveID                       1140 non-null   object 
 1   vendorProject               1140 non-null   object 
 2   product                     1140 non-null   object 
 3   vulnerabilityName           1140 non-null   object 
 4   dateAdded                   1140 non-null   object 
 5   shortDescription            1140 non-null   object 
 6   requiredAction              1140 non-null   object 
 7   dueDate                     1140 non-null   object 
 8   knownRansomwareCampaignUse  1140 non-null   object 
 9   notes                       373 non-null    object 
 10  cwes                        0 non-null      float64
dtypes: float64(1), object(10)
memory usage: 98.1+ KB


In [6]:
# select numeric
len(df.select_dtypes("number").columns) # count of numeric columns

1

In [7]:
#select object columns
len(df.select_dtypes("object").columns) #count of object columns

10

In [8]:
df.describe()

,cwes
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


**What I see:**


- I have 1140 rows and 11columns
- 1 of the 11 columns are numric while 10 of them are string object colums
- the numeric column `cwes` is null
- the `notes` column has 373 non null rows
- descriptive statistics says nothing on the `cwes` empty column

In [9]:
# remove the non useul unique id and the emplt column
df = df.drop(columns=["cveID", "dateAdded","cwes"])
df.columns

Index(['vendorProject', 'product', 'vulnerabilityName', 'shortDescription',
       'requiredAction', 'dueDate', 'knownRansomwareCampaignUse', 'notes'],
      dtype='object')

In [10]:
# nanipulate the column names to a clear name
spaced_cols = []
col_renamed = ""
for name in df.columns:
    for ele in name:
        if ele == ele.upper():
            col_renamed += " " + ele
        else:
            col_renamed += ele
    spaced_cols.append(col_renamed)
    col_renamed = ""

df.columns = spaced_cols
spaced_cols

['vendor Project',
 'product',
 'vulnerability Name',
 'short Description',
 'required Action',
 'due Date',
 'known Ransomware Campaign Use',
 'notes']

In [11]:
# remove spaces from the ends
# replace " " with _
# convert from lower case
new_cols = df.columns.str.strip().str.replace(" ", "_").str.lower()
df.columns = new_cols #replace the original columns with the new columns
df.columns

Index(['vendor_project', 'product', 'vulnerability_name', 'short_description',
       'required_action', 'due_date', 'known_ransomware_campaign_use',
       'notes'],
      dtype='object')

In [12]:
# remove duplicated rows
print("original data size: ", df.shape)
df = df.drop_duplicates(keep="first")
print("new data size: ", df.shape)

original data size:  (1140, 8)
new data size:  (1099, 8)


In [13]:
# Split the data into 3 sets
train, validate, test = wrangle_.split_data_(df=df, 
                     test_size=0.2, 
                     validate_size= 0.2, 
                     stratify_col="known_ransomware_campaign_use", 
                     random_state=10)
train.shape, validate.shape, test.shape

((659, 8), (220, 8), (220, 8))

In [14]:
# reset index
train = train.reset_index(drop=True)

In [15]:
# count values for each feature
for col in train.columns:
    print(train[col].value_counts)
    print("unique value count size:",train[col].value_counts().shape)
    print("\n\n")

<bound method IndexOpsMixin.value_counts of 0          Zimbra
1          Google
2      SolarWinds
3         Mozilla
4           Rails
          ...    
654         Adobe
655     Hikvision
656     Microsoft
657         Cisco
658        Google
Name: vendor_project, Length: 659, dtype: object>
unique value count size: (121,)



<bound method IndexOpsMixin.value_counts of 0                         Collaboration (ZCS)
1                                 Chromium V8
2                                      Serv-U
3                     Firefox and Thunderbird
4                               Ruby on Rails
                        ...                  
654                                ColdFusion
655               Security cameras web server
656                         Internet Explorer
657    IOS Software and Cisco IOS XE Software
658                               Chromium V8
Name: product, Length: 659, dtype: object>
unique value count size: (308,)



<bound method IndexOpsMixin.value_counts of 0

# Tasks from EDA

* The source of memory corruption is from 4 vendors, but it looks like it's happening on different products of these vendors, so I will go back inthe wrangle file and separate the product from the volunerability and create a additional column for specific vulnerabilities groupd by wording. (cell 7 - cell 15)

In [16]:
train = pd.read_csv("./data/00_temp_train_data.csv", index_col=0)
# create a copy of the training data
temp_train = train.copy()

In [ ]:
# # Privilege Escalation Vulnerability  
# search_terms = ["Privilege", "Escalation"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [ ]:
# # Memory Corruption Vulnerability
# search_terms = ["Memory", "Corruption"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [ ]:
# # Arbitrary File Upload Vulnerability
# search_terms = ["File", "Upload"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [ ]:
# # Remote/Arbitrary Code Execution Vulnerability 
# search_terms = ["Execution", "Remote", "Arbitrary"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [ ]:
# # Improper Access Control Vulnerability
# search_terms = ["Access", "Improper", "Control"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [ ]:
# # Command Injection Vulnerability
# search_terms = ["Injection", "Command", "SQL"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [ ]:
# # Use-After-Free Vulnerability 
# search_terms = ["Use-After", "Use-After-Free"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [ ]:
# # Deserialization of Untrusted Data Vulnerability
# search_terms = ["Deserialization", "Untrusted"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [ ]:
# # Type Confusion Vulnerability  
# search_terms = ["Type Confusion"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [ ]:
# # Path Traversal Vulnerability
# search_terms = ["Traversal", "path"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [ ]:
# # Buffer Overflow Vulnerability
# search_terms = ["Buffer", "Overflow", "Heap"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [ ]:
# # Denial of Service Vulnerability 
# search_terms = ["Denial", "Service"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [ ]:
# # Information Disclosure Vulnerability
# search_terms = ["Information", "Disclosure"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [ ]:
# # Out-of-Bounds Write Vulnerability
# search_terms = ["Out-of-Bounds", "Write", "Bounds"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [ ]:
# # Sandbox Bypass Vulnerability 
# search_terms = ["Sandbox", "Bypass"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [ ]:
# # Arbitrary File Upload Vulnerability
# search_terms = ["Unspecified"]
# pattern = '|'.join(search_terms)

# train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)]

#### I cam combine all of the obove conditions into one as I engineer a new feature for vulnerabilities

In [23]:
# Create a function that handles feature engineering for vulnerability names
def feature_eng_vul_names(data_frame):
    # Define the conditions
    conditions = [
        data_frame['vulnerability_name'].str.contains(r'Privilege|Escalation', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Memory|Corruption', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'File|Upload', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Remote Code', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Access|Improper|Control', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Injection|Command|SQL', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Use-After|Use-After-Free', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Deserialization|Untrusted', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Type Confusion', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Traversal|path', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Buffer|Overflow|Heap', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Denial|Service', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Information|Disclosure', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Out-of-Bounds|Write|Bounds', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Sandbox|Bypass', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Unspecified', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Spoofing', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Input|Validation', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Hard-Coded|Credentials', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Forgery|SSRF|Request|Server', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Scripting|XSS|Cross|Site', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Arbitrary Code', case=False, na=False)
    ]

    # Define the corresponding values for the new feature
    choices = [
        "Privilege Escalation Vulnerability",
        "Memory Corruption Vulnerability",
        "File Upload Vulnerability",
        "Remote Code Execution Vulnerability",
        "Improper Access Control Vulnerability",
        "Command Injection Vulnerability",
        "Use-After-Free Vulnerability",
        "Deserialization of Untrusted Data Vulnerability",
        "Type Confusion Vulnerability",
        "Path Traversal Vulnerability",
        "Buffer Overflow Vulnerability",
        "Denial of Service Vulnerability",
        "Information Disclosure Vulnerability",
        "Out-of-Bounds Write Vulnerability",
        "Sandbox Bypass Vulnerability",
        "Arbitrary File Upload Vulnerability",
        "Spoofing Vulnerabilities",
        "Input Validation Vulnerabilities",
        "Use of Hard-Coded Credentials Vulnerabilities",
        "Server-Side Request Forgery (SSRF) Vulnerabilities",
        "Cross-Site Scripting (XSS) Vulnerabilities",
        "Arbitrary Code Execution Vulnerability",
    ]

    # Use np.select to create the new feature
    data_frame['vulnerability_type'] = np.select(conditions, choices, default="Other Vulnerability")
    return data_frame


#### The newly engineered feature values

In [24]:
feature_eng_vul_names(train).vulnerability_type.value_counts()

vulnerability_type
Privilege Escalation Vulnerability                    93
Remote Code Execution Vulnerability                   90
Command Injection Vulnerability                       59
Improper Access Control Vulnerability                 46
Sandbox Bypass Vulnerability                          43
Memory Corruption Vulnerability                       39
Buffer Overflow Vulnerability                         39
Use-After-Free Vulnerability                          35
Other Vulnerability                                   34
Path Traversal Vulnerability                          24
Information Disclosure Vulnerability                  23
File Upload Vulnerability                             20
Out-of-Bounds Write Vulnerability                     18
Denial of Service Vulnerability                       18
Deserialization of Untrusted Data Vulnerability       16
Type Confusion Vulnerability                          15
Arbitrary File Upload Vulnerability                   12
Cross-Site S

* Apply data spliting here ater preping is done.

In [25]:
feature_eng_vul_names(df)

,vendor_project,product,vulnerability_name,short_description,required_action,due_date,known_ransomware_campaign_use,notes,vulnerability_type
0,Accellion,FTA,Accellion FTA OS Command Injection Vulnerability,Accellion FTA contains an OS command injection...,Apply updates per vendor instructions.,2021-11-17,Known,NaN,Command Injection Vulnerability
1,Accellion,FTA,Accellion FTA OS Command Injection Vulnerability,Accellion FTA contains an OS command injection...,Apply updates per vendor instructions.,2021-11-17,Known,NaN,Command Injection Vulnerability
2,Accellion,FTA,Accellion FTA SQL Injection Vulnerability,Accellion FTA contains a SQL injection vulnera...,Apply updates per vendor instructions.,2021-11-17,Known,NaN,Command Injection Vulnerability
3,Accellion,FTA,Accellion FTA Server-Side Request Forgery (SSR...,Accellion FTA contains a server-side request f...,Apply updates per vendor instructions.,2021-11-17,Known,NaN,Server-Side Request Forgery (SSRF) Vulnerabili...
4,Adobe,Acrobat and Reader,Adobe Acrobat and Reader Heap-based Buffer Ove...,Acrobat Acrobat and Reader contain a heap-base...,Apply updates per vendor instructions.,2021-11-17,Unknown,NaN,Buffer Overflow Vulnerability
...,...,...,...,...,...,...,...,...,...
1135,Microsoft,Internet Explorer,Microsoft Internet Explorer Use-After-Free Vul...,Microsoft Internet Explorer contains a use-aft...,The impacted product is end-of-life and should...,2024-08-13,Unknown,https://learn.microsoft.com/en-us/lifecycle/pr...,Use-After-Free Vulnerability
1136,Acronis,Cyber Infrastructure (ACI),Acronis Cyber Infrastructure (ACI) Insecure De...,Acronis Cyber Infrastructure (ACI) allows an u...,Apply mitigations per vendor instructions or d...,2024-08-19,Unknown,https://security-advisory.acronis.com/advisori...,Other Vulnerability
1137,ServiceNow,"Utah, Vancouver, and Washington DC Now",ServiceNow Incomplete List of Disallowed Input...,"ServiceNow Washington DC, Vancouver, and earli...",Apply mitigations per vendor instructions or d...,2024-08-19,Unknown,https://support.servicenow.com/kb?id=kb_articl...,Denial of Service Vulnerability
1138,ServiceNow,"Utah, Vancouver, and Washington DC Now",ServiceNow Improper Input Validation Vulnerabi...,"ServiceNow Utah, Vancouver, and Washington DC ...",Apply mitigations per vendor instructions or d...,2024-08-19,Unknown,https://support.servicenow.com/kb?id=kb_articl...,Improper Access Control Vulnerability


In [26]:
# Split the data into 3 sets
train, validate, test = wrangle_.split_data_(df=df, 
                     test_size=0.2, 
                     validate_size= 0.2, 
                     stratify_col="known_ransomware_campaign_use", 
                     random_state=10)
train.shape, validate.shape, test.shape

((659, 9), (220, 9), (220, 9))